# 421a Tax Class Buildings 

This file pulls all 421a buildings in Queens and filters out for those in Assembly District 36. 

Using available information on 421a buildings from the NYC Department of Finance, we are using the most up-to-date list from 2021. I am then pulling all Borough-Block-Lot (BBL) locations available through NYC Department of Buildings through NYC OpenData. By pulling BBL, we can easily merge the Dept of Finance data with Department of Buildings data and get both 421a buildings and their locations. 

Finally, using NYC OpenData, I am able to spatial join the buildings data with NY State Assembly Districts to get buildings within AD36. The file used for ditrict boundaries before the NY state redistricting announced in early 2022. 

In [49]:
import pandas as pd
import numpy as np
import requests
from sodapy import Socrata
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

import geopandas as gpd
import geojson
import keplergl
import json
import openpyxl
import geopy
from shapely.geometry import Polygon, Point, MultiPolygon

## 421A Locations

Read in 421a locations and add BBL indicator. BBL is how NYC identifies building locations. It's Borough-Block-Lot, with further distinctions but we won't need those with this analysis.

Locations are from: https://www1.nyc.gov/site/finance/benefits/benefits-421a.page You can download any of the borough's 421A locations from this page. I use the 2020-21 data but they also have historical files. 

In [50]:
#If you'd like you can read directly from the 
url_loc = "https://www1.nyc.gov/assets/finance/downloads/pdf/421a/2021/421a_2021_queens.xlsx"
properties = pd.read_excel(url_loc, header = 4)


In [51]:
#Clean up BBL fields
properties['BLOCK'] = properties['BLOCK'].astype(str)
properties['BOROUGH'] = properties['BOROUGH'].astype(str)
properties['LOT'] = properties['LOT'].astype(str)

#Add leading zeros to complete BBL
#BBL format: 
properties['BLOCK'] = properties['BLOCK'].apply(lambda x: ("00000" + x)[-5:])
properties['LOT'] = properties['LOT'].apply(lambda x: ("0000" + x)[-4:])

#Merge
properties['BBL'] = properties['BOROUGH'] + properties['BLOCK'] + properties['LOT']

In [52]:
#clean up columns
cols = properties.columns
new_cols = [x.replace("\n", " ") for x in cols]
properties.columns = new_cols

In [53]:
print("We are looking at {} buildings in Queens using the 421a program.".format(properties.shape[0]))

We are looking at 17895 buildings in Queens using the 421a program.


## Read BBL Location Data

Pulling from NYC OpenData: https://opendata.cityofnewyork.us 



In [54]:
load_dotenv()

#load environment variables
API_Key = os.getenv('NYC_API_Key')
API_Secret_Key = os.getenv('NYC_API_Secret_Key')
App_token = os.getenv('NYC_App_token')

#Set up client
client = Socrata("data.cityofnewyork.us", app_token=App_token)

In [55]:
#pull any buildings where the bbl value starts with 4 (that's Queens borough indicator). Warning, this could take a while. Start with a small limit and test out your process first. 
astoria_df = client.get("7w4b-tj9d", where = "base_bbl like '4%'", limit=500000000)
# https://data.cityofnewyork.us/resource/cwab-e33n.json

In [56]:
#convert to a dataframe
astoria_df = pd.DataFrame.from_records(astoria_df)

In [57]:
#Extract Latitude and Longitude values from geometry
astoria_df['Location'] = astoria_df['the_geom'].apply(lambda x: x['coordinates'])
# astoria_df['Location'] = astoria_df['the_geom'].apply(lambda x: np.array(x['coordinates'][0][0]).mean(axis = 0))
astoria_df['Latitude'] = astoria_df['Location'].apply(lambda x: x[1])
astoria_df['Longitude'] = astoria_df['Location'].apply(lambda x: x[0])
astoria_df.drop('Location', inplace = True, axis = 1)
astoria_df['geometry'] = astoria_df[['Longitude', 'Latitude']].apply(Point, axis=1)

#Convert to geoDF
geo_astoria_df = gpd.GeoDataFrame(astoria_df)

In [58]:
geo_astoria_df.head()

,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,base_bbl,mpluto_bbl,geomsource,name,Latitude,Longitude,geometry
0,"{'type': 'Point', 'coordinates': [-73.75428671...",4161096,1950,2017-08-22T00:00:00.000,Constructed,746409,18.01511294,2100,93,4075020005,4075020005,Photogramm,NaN,40.755844,-73.754287,POINT (-73.75429 40.75584)
1,"{'type': 'Point', 'coordinates': [-73.81927032...",4559791,1920,2017-08-17T00:00:00.000,Constructed,1205350,12.1716,5110,40,4096030070,4096030070,Photogramm,NaN,40.685108,-73.819270,POINT (-73.81927 40.68511)
2,"{'type': 'Point', 'coordinates': [-73.82703672...",4199149,1925,2017-08-22T00:00:00.000,Constructed,607250,29.18,2100,55,4094370034,4094370034,Photogramm,NaN,40.692657,-73.827037,POINT (-73.82704 40.69266)
3,"{'type': 'Point', 'coordinates': [-73.78780150...",4155509,1945,2017-08-22T00:00:00.000,Constructed,562240,23.0719761,2100,60,4072290076,4072290076,Photogramm,NaN,40.726353,-73.787802,POINT (-73.78780 40.72635)
4,"{'type': 'Point', 'coordinates': [-73.74983012...",4586100,1925,2017-08-17T00:00:00.000,Constructed,1195315,12.42,5110,67,4109150018,4109150018,Photogramm,NaN,40.710095,-73.749830,POINT (-73.74983 40.71009)


In [59]:
### Read in Assembly districts data
AD_map = pd.read_csv('../data/gathered_data/nyad.csv')
AD_map.rename(columns={'the_geom': 'geometry'}, inplace = True)
geo_AD_map = gpd.GeoDataFrame(AD_map)
geo_AD_trial = gpd.GeoSeries.from_wkt(AD_map['geometry'])
geo_AD_map['geometry'] = geo_AD_trial

In [60]:
geo_AD_map.head()

,Shape_Leng,Shape_Area,AssemDist,geometry
0,74160.255714,9.285877e+07,58,"MULTIPOLYGON (((-73.91282 40.65878, -73.91242 ..."
1,40309.973923,5.508234e+07,43,"MULTIPOLYGON (((-73.93112 40.66880, -73.93113 ..."
2,51820.348482,7.374100e+07,48,"MULTIPOLYGON (((-73.96092 40.62767, -73.95996 ..."
3,66327.162217,1.322116e+08,32,"MULTIPOLYGON (((-73.75763 40.66645, -73.75773 ..."
4,64345.987490,7.835775e+07,87,"MULTIPOLYGON (((-73.85574 40.84314, -73.85473 ..."


In [61]:
#Join NYC buildings data with assembly districts
buildings_districts = geo_astoria_df.sjoin(geo_AD_map)

In [62]:
#Filter for 36th district
ad36_buildings = buildings_districts.loc[buildings_districts['AssemDist'] == 36]
ad36_buildings = pd.DataFrame(ad36_buildings)
ad36_buildings.drop(['geometry'], axis = 1, inplace = True)

In [63]:
print("We are dealing with {} buildings in AD36.".format(ad36_buildings.shape[0]))

We are dealing with 19009 buildings in AD36.


In [64]:
print(ad36_buildings.shape)
ad36_buildings.head(1)

(19009, 19)


,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,base_bbl,mpluto_bbl,geomsource,name,Latitude,Longitude,index_right,Shape_Leng,Shape_Area,AssemDist
78,"{'type': 'Point', 'coordinates': [-73.90749811...",4013515,1940,2017-08-22T00:00:00.000,Constructed,278238,25.34,2100,68,4007310012,4007310012,Photogramm,NaN,40.763277,-73.907498,30,71034.987493,1.058894e+08,36


## Merge 421A Properties with District 36 Properties

Now that we have both 421A properties and AD36 properties, we can merge these two on the BBL value and get 421A properties within the 36th district.

In [65]:
ad36_421Properties = ad36_buildings.merge(properties, left_on='base_bbl', right_on='BBL', how = 'inner')

In [66]:
ad36_421Properties.head()

,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,base_bbl,...,BUILDING CLASS,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,BBL
0,"{'type': 'Point', 'coordinates': [-73.92937020...",4006371,2009,2017-08-22T00:00:00.000,Constructed,1113607,100.19426021,2100,10,4005520022,...,D6,21-16 31 AVENUE,11106.0,32,1,33,8550,33500,2009,4005520022
1,"{'type': 'Point', 'coordinates': [-73.92547417...",4594935,2009,2017-08-22T00:00:00.000,Constructed,1112303,54.76824075,2100,35,4005780026,...,C1,25-16 30 DRIVE,11102.0,8,0,8,3669,10976,2009,4005780026
2,"{'type': 'Point', 'coordinates': [-73.93419912...",4005774,2004,2017-08-22T00:00:00.000,Constructed,990835,34.25,2100,12,4005190024,...,C0,31-41 12 STREET,11106.0,3,0,3,2846,4020,2004,4005190024
3,"{'type': 'Point', 'coordinates': [-73.91652317...",4010640,2008,2017-08-22T00:00:00.000,Constructed,1222252,60.80873571,2100,68,4006600024,...,D7,30-47 38 STREET,11103.0,6,1,7,2496,7575,2008,4006600024
4,"{'type': 'Point', 'coordinates': [-73.92104311...",4018535,2009,2017-08-22T00:00:00.000,Constructed,1218833,53.8536847,2100,29,4008610044,...,D1,25-22 HOYT AVENUE SOUTH,11102.0,8,0,8,2603,7700,2009,4008610044


In [67]:
ad36_421Properties.shape

(263, 35)

In [68]:
#Here we need to drop all the duplicates. This will be by address and the geomsource
# In the NYC buildings data, we get two types of values for the "geomsource" (Photogramm and ______). NYC Dept of Buildings updated info on many of these buildings leaving duplicates. Sorting by geomsource and dropping duplicates off of 
#   ADDRESS solves this.
ad36_421Properties.sort_values('geomsource', ascending=False).drop_duplicates(subset=['ADDRESS'])

,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,base_bbl,...,BUILDING CLASS,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,BBL
0,"{'type': 'Point', 'coordinates': [-73.92937020...",4006371,2009,2017-08-22T00:00:00.000,Constructed,1113607,100.19426021,2100,10,4005520022,...,D6,21-16 31 AVENUE,11106.0,32,1,33,8550,33500,2009,4005520022
157,"{'type': 'Point', 'coordinates': [-73.92678903...",4006731,2006,2017-08-22T00:00:00.000,Constructed,306788,52.81,2100,25,4005710011,...,D1,23-25 30 DRIVE,11102.0,20,0,20,7512,15675,2006,4005710011
144,"{'type': 'Point', 'coordinates': [-73.92192464...",4008479,2006,2017-08-22T00:00:00.000,Constructed,990867,50.56,2100,39,4006150023,...,C1,30-35 31 STREET,11102.0,8,0,8,2000,5465,2006,4006150023
145,"{'type': 'Point', 'coordinates': [-73.93027849...",4608063,2007,2017-08-17T00:00:00.000,Constructed,9067,7.55,5110,11,4005350029,...,C2,30-35 14 STREET,11102.0,5,0,5,2375,5736,2007,4005350029
147,"{'type': 'Point', 'coordinates': [-73.89366967...",4533611,2006,2017-08-22T00:00:00.000,Constructed,1062946,31.35568442,2100,52,4009450044,...,C0,19-67 76 STREET,11370.0,3,0,3,2250,3234,2006,4009450044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,"{'type': 'Point', 'coordinates': [-73.91482896...",4009440,2007,2017-08-22T00:00:00.000,Constructed,1085512,53.0,2100,57,4006330051,...,C1,25-34 36 STREET,11103.0,8,0,8,2462,7151,2007,4006330051
49,"{'type': 'Point', 'coordinates': [-73.92043589...",4007789,2012,2017-08-22T00:00:00.000,Constructed,1255434,40.0,2100,55,4005970120,...,C1,26-50 30 STREET,11102.0,7,0,7,2502,5000,2012,4005970120
50,"{'type': 'Point', 'coordinates': [-73.92364626...",4616533,2016,2017-08-22T00:00:00.000,Constructed,1279660,70.0,2100,39,4008720011,...,D9,23-15 ASTORIA BOULEVARD,11102.0,37,0,37,10276,38592,2013,4008720011
262,"{'type': 'Point', 'coordinates': [-73.92390459...",4007584,2004,2017-08-22T00:00:00.000,Constructed,988315,28.0,2100,43,4005900011,...,C2,30-74 30 STREET,11102.0,6,0,6,1852,3312,2004,4005900011


In [69]:
print("We have {} buildings in AD36 using the 421a tax abatement program".format(ad36_421Properties.shape[0]))

We have 263 buildings in AD36 using the 421a tax abatement program


### Split By Tax Class

There's a couple reasons I split by tax class here. 

1. Each tax class represents a very different type of building. Most luxury apartment buildings fall under 2B or 2, whereas smaller multi-unit buildings fall under 1. Breaking this analysis this way allows for a more like-for-like analysis.
2. Each tax class has a different property tax document. So for this analysis where we will be pulling those tax documents, splitting by tax class allows for an easier scraping/parsing. 

In [70]:
ad36_421Properties['TAX CLASS'].unique()

array([2, '2B', 1, '2A'], dtype=object)

In [71]:
ad36_421Properties['TAX CLASS'].value_counts()

2B    105
2      71
2A     44
1      43
Name: TAX CLASS, dtype: int64

In [72]:
#Loop through tax classes to create individual building files.
tax_classes = ad36_421Properties['TAX CLASS'].unique()

for i in tax_classes:
    curr_taxClass = ad36_421Properties.loc[ad36_421Properties['TAX CLASS'] == i]
    file_name = 'ad36_421Properties_TaxClass{}.csv'.format(i)
    curr_taxClass.to_csv(file_name)